# Factores predictores para el desarrollo de epilepsia en el accidente cerebrovascular isquémico

## Introducción 

El accidente cerebrovascular (ACV) es una de las causas más comunes de epilepsia del adulto, siendo la incidencia de Epilepsia post-ACV (EPA) de alrededor del 7%. Como factores de riesgo asociados, se ha reportado la gravedad y el tipo del ACV, la topografía, NIHSS al ingreso y desarrollo de crisis tempranas (<7 días). Objetivo: analizar los factores predictores de desarrollo de EPA en nuestra población. 

## Materiales y métodos

Estudio observacional de cohorte retrospectiva de pacientes con ACV isquémico evaluados en nuestra institución entre 2014 y 2019, mayores de 18 años, seguimiento mínimo de 1 año. Se excluyeron pacientes con antecedentes personales de epilepsia. Se evaluaron variables demográficas, clínicas e imagenológicas. 

## Resultados

Se incluyeron 133 hombres y 63 mujeres internados durante 3 días (mediana, CI: 2 - CS: 6), NIHSS al ingreso 2 (mediana, CI: 0 - CS: 3).  Durante el seguimiento el 7% tuvo diagnóstico de EPA (15/196) con mayor frecuencia de transformación hemorrágica (TH), crisis tempranas, internaciones más prolongadas, mayor NIHSS al ingreso  y antecedentes psiquiátricos previos al evento índice, comparado con el grupo sin EPA. Estas variables presentaron correlaciones leves con EPA (todos rho Spearman < 0.5) y en el análisis multivariable la presencia de TH (Odds ratio=35.4, p<0.05, IC 2.5%-97.5%: 6.69 - 187) y el tiempo total de internación (Odds ratio=0.62, p<0.05, IC 2.5%-97.5%: 0.54 - 0.72) fueron factores de riesgo independientes.

## Conclusiones

Este estudio destaca aspectos relevantes de la EPA en nuestra población. Las variables relacionadas con mayor riesgo de EPA fueron similares a las reportadas en la literatura mundial, destacándose la transformación hemorrágica y la mayor duración de internación. Los datos obtenidos servirán como fundamento para la realización de un modelo predictivo mediante la tecnología del aprendizaje automático, que permita individualizar la probabilidad de epilepsia post-ACV. 


# 1. Set up

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
# Set up
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns

df=pd.read_excel('/Users/fernandomarquez/Documents/Fleni/DF/Epilepsia post-stroke(2020-08-31).xlsx',
                  #parse_dates=['Fecha', 'Fecha egreso'],
                #  skipinitialspace=True,encoding="utf-8"
                )

df1= pd.read_excel('/Users/fernandomarquez/Documents/Fleni/DF/Registro Vascular 2016-2020 para trabajo epi postACV.xlsx',
                  parse_dates=['Fecha', 'Fecha egreso'],
                 # skipinitialspace=True,encoding="utf-8"
                  )

df3= pd.read_excel('/Users/fernandomarquez/Documents/Fleni/DF/Registro vascular Base 2015-2016 - UCV.xlsx',
                   header=1,
                  #skipinitialspace=True,encoding="utf-8"
                  )

df4=pd.read_excel('/Users/fernandomarquez/Documents/Fleni/DF/Epilepsia postACV busqueda SIDCA 03-09-2020.xls',
                  parse_dates=['Fecha de creación','Fecha de Modificacion'],
                 # skipinitialspace=True,encoding="utf-8"
    )

pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 100
pd.options.display.max_rows = 500
pd.set_option('display.float_format', lambda x: '%.3g' % x)

/Users/fernandomarquez/.local/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Revision de HC
## Split en 3 datasets

In [3]:
df.columns

Index(['Submission Date', 'Criterio', 'NHC', 'NCH equivocada?',
       'Fecha de ingreso',
       'Tiene el paciente algún control neurólogico en Fleni posterior al ACV índice?',
       'Fecha del último control neurológico en Fleni',
       'Tiempo desde el evento índice al seguimiento',
       'Durante el seguimiento, en algún control se le diagnosticó Epilepsia post-stroke?',
       'Tuvo el paciente alguno de los siguientes criterios de exclusión?',
       ...
       'ACVi localización >> Talamo >> Bilateral',
       'ACVi localización >> Insula >> NR',
       'ACVi localización >> Insula >> Derecho',
       'ACVi localización >> Insula >> Izquierdo',
       'ACVi localización >> Insula >> Bilateral', 'Hipocampo', 'Fazekas',
       'Ver escala de Fazekas', 'Email', 'Email Maxi'],
      dtype='object', length=272)

In [4]:
df[['Criterio',
 'Fecha de ingreso',
 'Tiene el paciente algún control neurólogico en Fleni posterior al ACV índice?',
 'Fecha del último control neurológico en Fleni',
# 'Tiempo desde el evento índice al seguimiento',
 'Durante el seguimiento, en algún control se le diagnosticó Epilepsia post-stroke?',
 'Tuvo el paciente alguno de los siguientes criterios de exclusión?',]].describe()

/Users/fernandomarquez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  import sys
/Users/fernandomarquez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  import sys


,Criterio,Fecha de ingreso,Tiene el paciente algún control neurólogico en Fleni posterior al ACV índice?,Fecha del último control neurológico en Fleni,"Durante el seguimiento, en algún control se le diagnosticó Epilepsia post-stroke?",Tuvo el paciente alguno de los siguientes criterios de exclusión?
count,763,767,767,622,612,52
unique,2,607,2,379,2,12
top,No incluido,2016-07-28 00:00:00,Si,2020-07-20 00:00:00,No,Antecedentes personales de epilepsia
freq,402,5,622,8,589,16
first,NaN,1901-01-01 00:00:00,NaN,2015-05-11 00:00:00,NaN,NaN
last,NaN,2020-07-01 00:00:00,NaN,2020-10-30 00:00:00,NaN,NaN


In [5]:
df.Criterio

0      No incluido
1         Incluido
2      No incluido
3         Incluido
4      No incluido
          ...     
762    No incluido
763       Incluido
764    No incluido
765       Incluido
766    No incluido
Name: Criterio, Length: 767, dtype: object

In [6]:
df.loc[(df['Criterio'] == 'Incluido')&
       (df['Tiempo desde el evento índice al seguimiento']>365)
      # (df['Criterio'] == 'No incluido')&
      ]#.shape

,Submission Date,Criterio,NHC,NCH equivocada?,Fecha de ingreso,Tiene el paciente algún control neurólogico en Fleni posterior al ACV índice?,Fecha del último control neurológico en Fleni,Tiempo desde el evento índice al seguimiento,"Durante el seguimiento, en algún control se le diagnosticó Epilepsia post-stroke?",Tuvo el paciente alguno de los siguientes criterios de exclusión?,Si querés que te llegue un mail con el link de edición seleccioná tu nombre,Fecha de nacimiento,Edad al ingreso,Género,Preferencia manual,Antecedentes personales clínicos,Historia familiar de epilpesia,HTA,DM tipo I,DM tipo II,Dislipemia,Arritmia,Tipo arritmia,Cardiopatía isquemica,ICC,ACVi,ACVh,AIT,Epilepsia,Deterioro cognitivo,Demencia,Hipotiroidismo,EPOC,Asma,Inmunológicas,Cancer,Tipo cáncer,Trastornos psiquiátricos,Tipo antecedente psiquiátrico,Trastornos de aprendizaje,Trastornos de conducta,SAHOS,Otros antecedentes,Antecedente de neurocirugía,Tipo de neurocirugía,Hábitos toxicos,Consumo regular de tabaco previo al evento,Pack/year,Consumo de alcohol previo al evento,Frecuencia de consumo de alcohol previo al evento,...,ACVi localización >> Parietal >> Bilateral,ACVi localización >> Occipital >> NR,ACVi localización >> Occipital >> Derecho,ACVi localización >> Occipital >> Izquierdo,ACVi localización >> Occipital >> Bilateral,ACVi localización >> Capsula interna >> NR,ACVi localización >> Capsula interna >> Derecho,ACVi localización >> Capsula interna >> Izquierdo,ACVi localización >> Capsula interna >> Bilateral,ACVi localización >> Cerebelo >> NR,ACVi localización >> Cerebelo >> Derecho,ACVi localización >> Cerebelo >> Izquierdo,ACVi localización >> Cerebelo >> Bilateral,ACVi localización >> Ganglios de la base >> NR,ACVi localización >> Ganglios de la base >> Derecho,ACVi localización >> Ganglios de la base >> Izquierdo,ACVi localización >> Ganglios de la base >> Bilateral,ACVi localización >> Medula >> NR,ACVi localización >> Medula >> Derecho,ACVi localización >> Medula >> Izquierdo,ACVi localización >> Medula >> Bilateral,ACVi localización >> Mesencefalo >> NR,ACVi localización >> Mesencefalo >> Derecho,ACVi localización >> Mesencefalo >> Izquierdo,ACVi localización >> Mesencefalo >> Bilateral,ACVi localización >> Protuberancia >> NR,ACVi localización >> Protuberancia >> Derecho,ACVi localización >> Protuberancia >> Izquierdo,ACVi localización >> Protuberancia >> Bilateral,ACVi localización >> Bulbo >> NR,ACVi localización >> Bulbo >> Derecho,ACVi localización >> Bulbo >> Izquierdo,ACVi localización >> Bulbo >> Bilateral,ACVi localización >> Limítrofe >> NR,ACVi localización >> Limítrofe >> Derecho,ACVi localización >> Limítrofe >> Izquierdo,ACVi localización >> Limítrofe >> Bilateral,ACVi localización >> Talamo >> NR,ACVi localización >> Talamo >> Derecho,ACVi localización >> Talamo >> Izquierdo,ACVi localización >> Talamo >> Bilateral,ACVi localización >> Insula >> NR,ACVi localización >> Insula >> Derecho,ACVi localización >> Insula >> Izquierdo,ACVi localización >> Insula >> Bilateral,Hipocampo,Fazekas,Ver escala de Fazekas,Email,Email Maxi
1,2020-08-23 18:19:50,Incluido,593367,NaN,2016-12-12,Si,2019-12-05,1.09e+03,No,NaN,Abril,1945-09-24,71.3,Masculino,Diestro,HTA,No,No,No,No,No,No,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN,No,NaN,No,No,No,NaN,No,NaN,No,NaN,nan,NaN,NaN,...,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,NaN,nan,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,nan,NaN,nan,NaN,abrilmarone@gmail.com,NaN
3,2020-08-23 18:16:56,Incluido,593118,NaN,2016-12-06,Si,2019-12-06,1.1e+03,No,NaN,Abril,1958-11-24,58.1,Masculino,Diestro,DLP\nCI\nPsiquiátricos,No,No,No,No,Si,No,NaN,Si,No,No,No,No,No,No,No,No,No,No,No,No,NaN,Si,Trastorno de ansiedad,No,No,No,NaN,No,NaN,Si,Si previo,30,No,NaN,...,NaN,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,NaN,nan,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,nan,NaN,

In [7]:
df.loc[df['Tuvo el paciente alguno de los siguientes criterios de exclusión?']]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [ ]:
df.loc[(~df['crieterio_exclusion'].isna().sum()) #& 
       #(df['Criterio']=='Incluido')
      ]

In [ ]:
df.criterio_exclusion

### Split primer registro vascular

In [ ]:
## Select columns to split
#df_split=df[['HC', 'Fecha', 'Edad', 'Sexo', 'TOAST', 'ESUS','NIH ingreso', 'mRS previo ingreso','Sala internacion', 'Tratamiento', 
 #            'r-TPA','AAG/ACO al alta', 'NIH egreso', 'mrS egreso', 'Fecha egreso', 'Días UCV','Días UTI', 'Días Piso',
  #           'obito']].sort_values(by=['HC','Fecha'])

## Replace NaN with string
#df_split= df_split.replace(np.nan, 'COMPLETAR', regex=True)

## Split dataframe in 3chunks 
#num_chunks = 3
#df1,df2,df3 = np.array_split(df_split,num_chunks)

# Define function to color values by condition
#def _color_red_or_green(val):
 #   color = 'red' if val == 'COMPLETAR' else 'green'
  #  return 'color: %s' % color

# Apply style to dataframe with fuction to color and Save chunks
#df1.style.applymap(_color_red_or_green).to_excel('revision_mati.xlsx')
#df2.style.applymap(_color_red_or_green).to_excel('revision_abril.xlsx')
#df3.style.applymap(_color_red_or_green).to_excel('revision_fer.xlsx')

### Split del segundo registro vascular

In [ ]:
## Select columns to split
#df_split=df3.loc[(df3['Fecha ingreso'] != '2016') | (df3['Fecha ingreso'] != '2017')]
## Replace NaN with string
#df_split= df_split.replace(np.nan, 'COMPLETAR', regex=True)

## Split dataframe in 3chunks 
#num_chunks = 3
#df1,df2,df3 = np.array_split(df_split,num_chunks)

# Define function to color values by condition
#def _color_red_or_green(val):
 #   color = 'red' if val == 'COMPLETAR' else 'green'
  #  return 'color: %s' % color

# Apply style to dataframe with fuction to color and Save chunks
#df3.style.applymap(_color_red_or_green).to_excel('revision2_fer.xlsx')

#df1.to_excel('revision2_mati.xlsx')
#df2.to_excel('revision2_abril.xlsx')
#df3.to_excel('revision2_fer.xlsx')

## Busqueda SIDCA

In [ ]:
#df2014_2015=df4.loc[df4['Fecha de creación'] >  '2013-12-30 00:00:00']
#df2014_2015.to_excel('sidca_2014_2015.xlsx')

In [ ]:
#df2012_2013=df4.loc[df4['Fecha de creación'] <  '2014-01-01 00:00:00']
#df2012_2013.to_excel('sidca_2012_2013.xlsx')

# Preprocesado

## Renombrar columnas

In [12]:
# cambiar mayusculas a minusculas los nombres de las columnas
df1.columns = map(str.lower, df1.columns)

In [13]:
# crear un diccionario para los nombres de las columnas
dict_cols= {'hc':'NHC',
            'fecha':'fecha_ingreso',
            'sexo':'genero',
            'dx principal':'dx',
            'toast':'TOAST',
            'esus':'ESUS',
            'especificación dx':'observaciones',
            'nih ingreso':'NIHSSi',
            'mrs previo ingreso':'mRSi',
            'sala internacion':'sala_inicio',
            'ingreso en ventana (rtpa)':'ventana',
            'tratamiento':'tipo_tto',
            'r-tpa':'rTPA',
            'trop t':'TnT',
            'cancer asociado':'ca_asociado',
            'aag/aco al alta':'prev2_alta',
            'cierre fop?':'cierre_FOP',
            'nih egreso':'NIHSSe',
            'mrs egreso':'mRSe',
            'fecha egreso':'fecha_egreso',
            'días totales':'diasTotal',
            'días ucv':'diasUCV',
            'días uti':'diasUTI',
            'días piso':'diasPiso'}
            
# renombrar columnas
df1=df1.rename(columns=dict_cols)

In [14]:
# crear diccionario para columnas
dict_cols2= {'Criterio':'criterio',
             'Fecha de ingreso':'ingreso',
             'Tiene el paciente algún control neurólogico en Fleni posterior al ACV índice?':'control',
             'Fecha del último control neurológico en Fleni':'control_fecha',
             'Tiempo desde el evento índice al seguimiento':'tiempo_seguimiento',
             'Durante el seguimiento, en algún control se le diagnosticó Epilepsia post-stroke?':'epilepsia_postACV',
             'Tuvo el paciente alguno de los siguientes criterios de exclusión?':'criterio_exclusion',
             'Si querés que te llegue un mail con el link de edición seleccioná tu nombre':'revisor',
             'Fecha de nacimiento':'nacimiento_fecha',
             'Edad al ingreso':'edad',
             'Género':'genero',
             'Preferencia manual':'preferencia_manual',
             'Antecedentes personales clínicos':'APP',
             'Historia familiar de epilpesia':'ant_fliar_epilepsia',
             'DM tipo I':'DMI',
             'DM tipo II':'DMII',
             'Dislipemia':'DLP',
             'Arritmia':'arritmia',
             'Tipo arritmia':'arritmia_tipo',
             'Cardiopatía isquemica':'CI',
             'Epilepsia':'ant_personal_epilepsia',
             'Deterioro cognitivo':'DCL',
             'Demencia':'demencia',
             'Hipotiroidismo':'hipotiroidismo',
             'EPOC':'EPOC',
             'Asma':'asma',
             'Inmunológicas':'inmunologicas',
             'Cancer':'CA',
             'Tipo cáncer':'CA_tipo',
             'Trastornos psiquiátricos':'psiquiatricos',
             'Tipo antecedente psiquiátrico':'psiquiatricos_tipo',
             'Trastornos de aprendizaje':'trastorno_aprendizaje',
             'Trastornos de conducta':'trastorno_conducta',
             'Otros antecedentes':'otros_APP',
             'Antecedente de neurocirugía':'neuroqx',
             'Tipo de neurocirugía':'neuroqx_tipo',
             'Hábitos toxicos':'toxicos',
             'Consumo regular de tabaco previo al evento':'TBQ',
             'Pack/year':'pack/year',
             'Consumo de alcohol previo al evento':'alcohol',
             'Frecuencia de consumo de alcohol previo al evento':'alcohol_frecuencia',
             'Cantidad de litros por día':'alcohol_litros',
             'Consumo regular de drogas previo al evento':'drogas',
             'Tipo de droga':'drogas_tipo',
             'Estatinas previo al evento índice':'estatina_previa',
             'Tipo y dosis de estatina >> Atorvastatina >> Si':'atorvavastina_previa',
             'Tipo y dosis de estatina >> Atorvastatina >> Dosis utilizada en mg':'atorvastatina_previa_dosis',
             'Tipo y dosis de estatina >> Pravastatina >> Si':'pravastatina_previa',
             'Tipo y dosis de estatina >> Pravastatina >> Dosis utilizada en mg':'pravastatina_previa_dosis',
             'Tipo y dosis de estatina >> Rosuvastatina >> Si':'rosuvastatina_previa',
             'Tipo y dosis de estatina >> Rosuvastatina >> Dosis utilizada en mg':'rosuvastatina_previa_dosis',
             'Tipo y dosis de estatina >> Sinvastatina >> Si':'sinvastatina_previa',
             'Tipo y dosis de estatina >> Sinvastatina >> Dosis utilizada en mg':'sinvastatina_previa_dosis',
             'Fecha de ACV':'ACV_fecha',
             'Fecha de egreso':'egreso',
             'Duración total internación':'internacion_dias',
             'Internación >> Guardia >> Inicial':'guardia_inicial',
             'Internación >> Guardia >> 2da sala':'guardia_2da_sala',
             'Internación >> Guardia >> 3er sala':'guardia_3er_sala',
             'Internación >> Guardia >> Días internación':'guardia_dias',
             'Internación >> UTI >> Inicial':'UTI_inicial',
             'Internación >> UTI >> 2da sala':'UTI_2da_sala',
             'Internación >> UTI >> 3er sala':'UTI_3er_sala',
             'Internación >> UTI >> Días internación':'UTI_dias',
             'Internación >> UCV >> Inicial':'UCV_inicial',
             'Internación >> UCV >> 2da sala':'UCV_2da_sala',
             'Internación >> UCV >> 3er sala':'UCV_3er_sala',
             'Internación >> UCV >> Días internación':'UCV_dias',
             'Internación >> Telemetría  >> Inicial':'telemetria_inicial',
             'Internación >> Telemetría  >> 2da sala':'telemteria_2da_sala',
             'Internación >> Telemetría  >> 3er sala':'telemtria_3er_sala',
             'Internación >> Telemetría  >> Días internación':'telemetria_dias',
             'Internación >> Sala >> Inicial':'sala_inicial',
             'Internación >> Sala >> 2da sala':'sala_2da_sala',
             'Internación >> Sala >> 3er sala':'sala_3er_sala',
             'Internación >> Sala >> Días internación':'sala_dias',
             'Etiología ACV isquémico (TOAST)':'TOAST',
             'TOAST otras causas':'TOAST_otras',
             'NIHSS ingreso':'NIHSSi',
             'mRS previo al ingreso':'mRSi',
             'Trombolítico':'r_TPA',
             'Trombectomía':'trombectomia',
             'Transformación hemorrágica':'transformacion_hemorragica',
             'Crisis epilépticas tempranas (dentro de los 7 días)':'crisis_tempranas',
             'DAE 1 en crisis tempranas':'crisis_temprana_DAE1',
             'DAE 2 en crisis tempranas':'crisis_temprana_DAE2',
             'Craniectomía descompresiva':'craniectomía',
             'Fecha de craniectomía':'craniectomia_fecha',
             'Craneoplastía':'craneoplastia',
             'Fecha de craneoplastía':'fecha_craneopalstia',
             'Hidrocefalia':'hidrocefalia',
             'Válvula de derivación':'hidrocefalia_valvula',
             'Fecha de válvula de derivación':'hidrocefalia_valvula_fecha',
             'NIHSS alta':'NIHSSe',
             'mRS alta':'mRSe',
             'Antiagregante 1':'antiagregante1',
             'Antiagregante 1 2':'antiagregante2',
             'Anticoagulante 1':'ACO1',
             'Anticoagulante 2':'ACO2',
             'Estatinas indicadas >> Atorvastatina >> Si':'atorvastatina_evento',
             'Estatinas indicadas >> Atorvastatina >> Dosis utilizada en mg':'atorvastatina_evento_dosis',
             'Estatinas indicadas >> Pravastatina >> Si':'pravastatina_evento',
             'Estatinas indicadas >> Pravastatina >> Dosis utilizada en mg':'pravastatina_evento_dosis',
             'Estatinas indicadas >> Rosuvastatina >> Si':'rosuvastatina_evento',
             'Estatinas indicadas >> Rosuvastatina >> Dosis utilizada en mg':'rosuvastatina_evento_dosis',
             'Estatinas indicadas >> Sinvastatina >> Si':'sinvastatina_evento',
             'Estatinas indicadas >> Sinvastatina >> Dosis utilizada en mg':'sinvastatina_evento_dosis',
             'Se le indicó DAE a pesar de no tener epilepsia?':'DAE_sin_epilepsia_evento',
             'DAE 1':'DAE1_evento',
             'DAE 2':'DAE2_evento',
             'DAE 3':'DAE3_evento',
             'Otros al alta >> Alprazolam >> Si':'alprazolam_evento',
             'Otros al alta >> Ciclofosfamida >> Si':'ciclofosfamida_evento',
             'Otros al alta >> Escitalopram >> Si':'escitalopram_evento',
             'Otros al alta >> Fluoxetina >> Si':'fluoxetina_evento',
             'Otros al alta >> Levodopa >> Si':'levodopa_evento',
             'Otros al alta >> Meprednisona >> Si':'meprednisona_evento',
             'Otros al alta >> Metilfenidato >> Si':'metilfenidato_evento',
             'Otros al alta >> Modafinilo >> Si':'modafinilo_evento',
             'Otros al alta >> Nimodipina >> Si':'nimodipina_evento',
             'Otros al alta >> Prednisona >> Si':'prednisona_evento',
             'Otros al alta >> Sertralina >> Si':'sertralina_evento',
             'Otros al alta >> Solumedrol >> Si':'solumedrol_evento',
             'Obito durante el evento idnice':'otros_evento',
             'Fecha de la consulta en la cual se realizó el diagnóstico de Epilepsia post-stroke':'epilepsia_fecha',
             'Tuvo en algún momento estatus epiléptico luego del evento índice?':'status',
             'Tipo de crisis':'tipo_crisis',
             'Frecuencia de crisis tradías':'frecuencia_crisis',
             'Se le indicó DAE a pesar de no tener epilepsia? 2':'DAE_sin_epilepsia_postACV',
             'DAE 1 en el seguimiento':'DAE1_postACV',
             'DAE 2 en el seguimiento':'DAE2_postACV',
             'DAE 3 en el seguimiento':'DAE3_postACV',
             'Otros durante el seguimiento >> Alprazolam >> Si':'alprazolam_postACV',
             'Otros durante el seguimiento >> Baclofeno >> Si':'baclofeno_postACV',
             'Otros durante el seguimiento >> Ciclofosfamida >> Si':'ciclofosfamida_postACV',
             'Otros durante el seguimiento >> Escitalopram >> Si':'escitalopram_postACV',
             'Otros durante el seguimiento >> Fluoxetina >> Si':'fluoxetina_postACV',
             'Otros durante el seguimiento >> Levodopa >> Si':'levodopa_postACV',
             'Otros durante el seguimiento >> Meprednisona >> Si':'meprednisona_postACV',
             'Otros durante el seguimiento >> Metilfenidato >> Si':'metilfenidato_postACV',
             'Otros durante el seguimiento >> Modafinilo >> Si':'modafinilo_postACV',
             'Otros durante el seguimiento >> Nimodipina >> Si':'nimodipina_postACV',
             'Otros durante el seguimiento >> Prednisona >> Si':'prednisona_postACV',
             'Otros durante el seguimiento >> Sertralina >> Si':'sertralina_postACV',
             'Otros durante el seguimiento >> Solumedrol >> Si':'solumedrol_postACV',
             'Obito posterior el evento idnice':'otros_postACV',
             'Se le realizó EEG durante el seguimiento?':'EEG',
             'Resultado del primer EEG realizado (o en su defecto alguno posterior)':'EEG_resultado',
            }

#Rename columns
df=df.rename(columns=dict_cols2)

In [15]:
df.tiempo_seguimiento.isnull().sum()

145

In [16]:
df.shape

(767, 272)

In [ ]:
# FP 145 sin seguimiento en fleni (767-145=622)

## Merge

763 HC revisadas (en ese momento)

No incluidos (n=402):
-145 no tenían control en Fleni
-52 cumplían criterios de exclusión
-205 no tenían seguimiento completo

Incluidos (n=196):
- 361 cumplían criterios de inclusión
- 67 eliminados por no tener Neuroimagen revisado (en ese momento) o con TAC (sin Fazekas y descripcion del infarto)
-170 registros de pacientes duplicados

In [ ]:
stop

In [10]:
df[df['criterio']=='Incluido'].shape

(361, 272)

In [17]:
# merge de datasets bajo el criterio de incluidos
df=df[df['criterio']=='Incluido'].merge(df1, on=['NHC'], how='inner')

In [18]:
df.shape

(357, 297)

## Duplicados

In [ ]:
df.shape

In [ ]:
df.loc[df.epilepsia_postACV == 'Si'][[]]

## Seguimiento

In [ ]:
print('Total pacientes: ', len(df))
print('Total de pacientes sin consulta postinternación en Fleni: ', df.tiempo_seguimiento.isnull().sum())
print()
print('Estadísticos de seguimiento') 
df.tiempo_seguimiento.describe()

In [ ]:
df.loc[df.epilepsia_postACV == 'Si'].sort_values(by='NHC')['NHC'].value_counts()

## Criterios de inclusion
- Edad igual o mayor a 18 años
- Diagnóstico clínico de Accidente Cerebrovascular (ACV) isquémico o hemorrágico (HIP) apoyado por una neuroimagen (Tomografía Computada o Resonancia Magnética de cerebro) 
- Diagnóstico de Epilepsia post-ACV de acuerdo a los criterios de la Liga Internacional contra la Epilepsia definido operativamente como crisis no provocadas (al menos una crisis siete días luego del evento índice).  
- Seguimiento completo de dos años, definido operativamente como el registro de: 
    - al menos un control neurológico dentro de los dos años desde el ACV en el que se haya diagnosticado epilepsia post-ACV o, 
    - al menos un control neurológico posterior a los dos años del ACV en el cual no se haya diagnosticado epilepsia poststroke

In [19]:
df.loc[df['criterio']=='Incluido'].shape

(357, 297)

In [20]:
# filtar pacientes por criterio de inclusión
index_incluidos= df['criterio']=='Incluido'
# crear array de NHC de pacientes incluidos
df=df.loc[index_incluidos]
print('Cantidad de pacientes que cumplen criterios de inclusión:', len(df))

Cantidad de pacientes que cumplen criterios de inclusión: 357


# Criterios de exclusión
- Antecedentes personales de epilepsia
- Seguimiento incompleto
- Antecedente de otra lesión cerebral estructural (malformación arteriovenosa, aneurisma, tumor, TEC, HSA)
- Trombosis venosa cerebral

In [21]:
# cantidad de pacientes que cumplen criterios de inclusion
print('Cantidad de pacientes que cumplen criterios de exclusión:',
      len(df)-len(df.loc[df['criterio_exclusion'].isnull()]))

Cantidad de pacientes que cumplen criterios de exclusión: 28


In [22]:
# cantidad de pacientes que cumplen criterios de inclusion
print('Cantidad de pacientes que cumplen criterios de exclusión:',
      len(df)-len(df.loc[df['criterio_exclusion'].isnull()]))
# eliminar pacientes que cumplen algun criterio de exclusión
df=df.loc[df['criterio_exclusion'].isnull()]
print('Total de pacientes incluidos:', len(df))

Cantidad de pacientes que cumplen criterios de exclusión: 28
Total de pacientes incluidos: 329


In [ ]:
df.loc[df.epilepsia_postACV == 'Si'].sort_values(by='NHC')['NHC'].value_counts()

In [ ]:
df.edad_x#.describe()

# Merge

In [23]:
# merge de datasets bajo el criterio de incluidos
df=df[df['criterio']=='Incluido'].merge(df1, on=['NHC'], how='inner')

In [24]:
df.shape

(433, 322)

# Eliminar columnas nulas

In [ ]:
df= df.dropna(axis=1, how='all')

In [ ]:
df.loc[]edad_x.describe()

In [ ]:
len(df)

# Fillna columnas imagenes

In [ ]:
#df.iloc[:, 137:267].columns

In [ ]:
#df=df.iloc[:, 137:267].apply(str)

In [ ]:
#df.dropna(axis=1, how='all').iloc[:, 137:267]
#df.loc[df.epilepsia_postACV == 'Si'].iloc[:, 137:]

In [ ]:
#list(zip(['ACVi territorio vascular >> ACA >> ']*4, ['NR', 'Derecho', 'Izquierdo', 'Bilateral']))

In [ ]:
#def normalise_row(row):
 #   if row['Currency'] != '$'
  #  ...
   # ...
   # ...
   # return result

#df['Normalized'] = df.apply(lambda row : normalise_row(row), axis=1) 

In [ ]:
#cols_ACA=['ACVi territorio vascular >> ACA >> NR',
 #         'ACVi territorio vascular >> ACA >> Derecho',
  #        'ACVi territorio vascular >> ACA >> Izquierdo',
   #       'ACVi territorio vascular >> ACA >> Bilateral']

#cols_Frontal=['ACVi localización >> Frontal >> NR',
 #         'ACVi localización >> Frontal >> Derecho',
  #        'ACVi localización >> Frontal >> Izquierdo',
   #       'ACVi localización >> Frontal >> Bilateral']
#df['combined'] = 
#df[cols_Frontal].sum(axis=1)

In [ ]:
# seleccionar las columnas de la tabla de doble entrada de imagenes
#df.iloc[:, 167:268].columns.to_list()

#df['ACVi territorio vascular >> ACA >> NR']+ df['ACVi territorio vascular >> ACA >> Derecho']+df['ACVi territorio vascular >> ACA >> Izquierdo']
#df['ACVi territorio vascular >> ACA >> Bilateral']
# completar los NaNs de tablas de doble entrada de imagenes
#df[df.iloc[:, 167:267].columns]=df.iloc[:, 167:267].fillna('No')

In [ ]:
#df[df.columns[df.isnull().all()]]

# Pacientes duplicados

## 1. Detectar reinternaciones

In [25]:
# Crear columna con los NHC duplicados
r = df.duplicated(['NHC'], keep=False)
df['reinternaciones'] = np.select([r],['si'], default='no')

#Crear columna con una o > reinternaciónes
r1=df[df['NHC'].isin(df['NHC'].value_counts()[df['NHC'].value_counts()==2].index)]
df['reinternacion1'] = 'no'
df.loc[df.NHC.isin(r1['NHC']), 'reinternacion1'] = 'si'

# Crear columna con mas de dos reinternaciónes
r2=df[df['NHC'].isin(df['NHC'].value_counts()[df['NHC'].value_counts()>2].index)]
df['reinternacion2'] = 'no'
df.loc[df.NHC.isin(r2['NHC']), 'reinternacion2'] = 'si'

# Ordenar por fecha de ingreso
df=df.sort_values(by=['ingreso'])

# Funcion para ver caracteristicas generales de la poblacion

def patients_characteristics (df):
    "Function to check patients characteristics"
    print('Total de pacientes (con duplicados):', len(df))
    print('Total de pacientes con Epilepsia PostACV (con duplicados):', len(df.loc[df['epilepsia_postACV']=='Si']))
    print('Total de pacientes con una o más reinternaciones:', len(df.loc[df['reinternaciones']=='si']))
    print('Total de pacientes con una única reinternación:', len(df.loc[df['reinternacion1'] == 'si']))
    print('Total de pacientes con dos o más reinternaciones:', len(df.loc[df['reinternacion2'] == 'si']))
    print('Total de pacientes duplicados:', len(df['NHC'])-len(df['NHC'].drop_duplicates()))
    return
patients_characteristics(df)

Total de pacientes (con duplicados): 433
Total de pacientes con Epilepsia PostACV (con duplicados): 26
Total de pacientes con una o más reinternaciones: 200
Total de pacientes con una única reinternación: 10
Total de pacientes con dos o más reinternaciones: 190
Total de pacientes duplicados: 170


## 2. Eliminar duplicados
Nos quedamos con el último registro y consideraremos el ACV como antecedente, para determinar si es una variable predictora

In [26]:
# ordenar el df por fecha de ingreso de forma descendente y quedarnos con el primer registo (última internación) 
df=df.sort_values(by=['ingreso']).drop_duplicates(subset=['NHC'], keep='first')
len(df)

263

# Valores erroneos 

In [ ]:
df.loc[(df.epilepsia_postACV=='Si')& (df.Fazekas.isnull())]['Neuroimagen de donde se obtiene la información']

# Eliminamos los que no han sido revisados por Maxi o han sido revisados pero tienen TAC

Aca se van 2 epilepsias post ACV

In [27]:
print('Total de pacientes sin Neuroimagen revisadoa o con TAC (sin Fazekas y descripcion del infarto)', 
      len(df.loc[(df['Cortical o subcortical?'].isnull())]))
print()
print('Total de pacientes con Epilepsia postStroke  sin Neuroimagen revisadoa o con TAC (sin Fazekas y descripcion del infarto)', 
      len(df.loc[(df['Cortical o subcortical?'].isnull())& (df['epilepsia_postACV']=='Si')]))

Total de pacientes sin Neuroimagen revisadoa o con TAC (sin Fazekas y descripcion del infarto) 67

Total de pacientes con Epilepsia postStroke  sin Neuroimagen revisadoa o con TAC (sin Fazekas y descripcion del infarto) 2


In [28]:
print('Total de pacientes sin Neuroimagen revisadoa o con TAC (sin Fazekas y descripcion del infarto)', 
      len(df.loc[(df['Transformación hemorrágica 2'].isnull())]))
print()
print('Total de pacientes con Epilepsia postStroke  sin Neuroimagen revisadoa o con TAC (sin Fazekas y descripcion del infarto)', 
      len(df.loc[(df['Transformación hemorrágica 2'].isnull())& (df['epilepsia_postACV']=='Si')]))

Total de pacientes sin Neuroimagen revisadoa o con TAC (sin Fazekas y descripcion del infarto) 64

Total de pacientes con Epilepsia postStroke  sin Neuroimagen revisadoa o con TAC (sin Fazekas y descripcion del infarto) 0


In [29]:
df.shape

(263, 325)

In [30]:
df=df.dropna(subset=['Cortical o subcortical?'])

In [31]:
print('Total de pacientes sin Neuroimagen revisadoa o con TAC (sin Fazekas y descripcion del infarto)', 
      len(df.loc[(df['Cortical o subcortical?'].isnull())]))

Total de pacientes sin Neuroimagen revisadoa o con TAC (sin Fazekas y descripcion del infarto) 0


In [32]:
def draw_missing_data_table(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data
draw_missing_data_table(df)

,Total,Percent
ACVi territorio vascular >> ACS >> Bilateral,196,1
ACVi territorio vascular >> AICA >> NR,196,1
ACVi territorio vascular >> Penetrantes >> Bilateral,196,1
ACVi territorio vascular >> Penetrantes >> Izquierdo,196,1
ACVi territorio vascular >> Penetrantes >> Derecho,196,1
ACVi territorio vascular >> Penetrantes >> NR,196,1
ACVi territorio vascular >> PICA >> Bilateral,196,1
ACVi territorio vascular >> PICA >> Izquierdo,196,1
ACVi territorio vascular >> PICA >> Derecho,196,1
ACVi territorio vascular >> PICA >> NR,196,1


## 1. Valores erróneos

In [ ]:
df= df.replace(['-','n/e','N/E','ne','NE','x','X','No corresponde','no corresponde','o', 'nan', 'NaN','Nan'],
               np.nan)

## Eliminamos las columnas con todos valores nulos

In [38]:
df.edad_x.describe()

count    187
mean    66.6
std     15.1
min     21.6
25%     59.2
50%     69.9
75%     77.3
max     93.5
Name: edad_x, dtype: float64

In [36]:
df.shape

(196, 325)

In [ ]:
## Eliminación 

df=df.dropna(axis=1, how='all')
df.shape

## 2. Variables numericas

In [ ]:
from funpymodeling.exploratory import cat_vars, num_vars
from funpymodeling.exploratory import status
from funpymodeling.exploratory import profiling_num


#status(df)

In [ ]:
num_vars(df)

In [ ]:
df[num_vars] = df[num_vars].astype(float)

In [ ]:
#df.loc[(df.diasUTI.isnull())|(df.diasPiso.isnull())|(df.diasUCV.isnull())].columns.to_list()

In [ ]:
df[['edad_x','NIHSSi_x','mRSi_x','NIHSSe_x','mRSe_x',
    'diasUCV_x','diasUTI_x', 'diasPiso_x','Fazekas','status',
 'tipo_crisis',
 'frecuencia_crisis',
 'DAE_sin_epilepsia_postACV',
 'DAE1_postACV','antiagregante1',
 'antiagregante2',
 'ACO1','sala_inicio_x', 'ESUS_x','r_TPA' ]].isnull().sum()

In [ ]:
# completar datos faltantes con valores del 2 dataframe
'edad_x','NIHSSi_x','mRSi_x','NIHSSe_x','mRSe_x'   
df[['edad_x']]=df['edad_x'].fillna(df['edad_y'])
df[['NIHSSi_x']]=df['NIHSSi_x'].fillna(df['NIHSSi_y'])
df[['NIHSSe_x']]=df['NIHSSe_x'].fillna(df['NIHSSe_y'])
df[['mRSi_x']]=df['mRSi_x'].fillna(df['mRSi_y'])
df[['mRSe_x']]=df['mRSe_x'].fillna(df['mRSe_y'])

df[['edad_x','NIHSSi_x','mRSi_x','NIHSSe_x','mRSe_x']].isnull().sum()

In [ ]:
df.loc[df['edad_x'].isnull()][['NHC','revisor','edad_x','mRSe_x']]

In [ ]:
df.loc[df['mRSe_x'].isnull()][['NHC','revisor','edad_x','mRSe_x']]

In [ ]:
num_vars(df)

In [ ]:
# variables numericas para convertir a str
#strcols_tonum=[]
# convertir columnas seleccionadas en objetos
#df[strcols_tonum] = df[strcols_tonum].applymap(lambda x: str(x))

## Eliminación 
Eliminamos las columnas con todos valores nulos

In [ ]:
#df=df.dropna(axis=1, how='all')

## Imputación
### 2.1. Valores nulos erroneos en variables numericas

In [ ]:
# estrategia 1: completar con mediana
num_cols_fillna_median=['edad_x','mRSe_x']
# 
df.loc[:, num_cols_fillna_median] = df.loc[:, num_cols_fillna_median].fillna(df.median)

In [ ]:
df[['edad_x','NIHSSi_x','mRSi_x','NIHSSe_x','mRSe_x']].isnull().sum()

In [ ]:
df.edad_x

### 2.2. Valores nulos equivalentes a 0 en variables numericas

In [ ]:
# definir lista con tipos de variables numericas
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
# crear lista de columns numericas
num_cols=df.select_dtypes(include=numerics).columns.values

In [ ]:
df[num_cols].isna().sum()

In [ ]:
# completar valores nulos equivalentes a 0
df.loc[:, num_cols] = df.loc[:, num_cols].fillna(0)

In [ ]:
df[num_cols].isna().sum()

## 3. Variables categóricas

# Regex

In [ ]:
import re

pattern = '^.{1,15}_y$'
#result = re.match(pattern, test_string)
cols=df.columns
cols_match=[]


for column in df[df.columns]:
    test_string=column
    result = re.match(pattern, test_string)
    if result:
        cols_match=np.append(cols_match,test_string)
    else:
        pass
cols_match

# Variables categoricas

In [ ]:
df.select_dtypes(include=['object']).columns.values

In [ ]:
df.select_dtypes(include=['object']).isnull().sum()

In [ ]:
str_cols=df.select_dtypes(include=['object']).columns#.values


#df[str_cols]=df.loc[df[str_cols]].fillna('No')
df.loc[:, str_cols] = df.loc[:, str_cols].fillna('No')

# Distribuciones

In [ ]:
from scipy.stats import shapiro

#Columnas a testear
shapiro_cols=df.select_dtypes(exclude=['datetime','object'])

# Crear dataframes vacios
dfs = []
df_nonormal=pd.DataFrame()
df_normal = pd.DataFrame()

for column in shapiro_cols:
   # seleccionar los contenidos de las columnas con el operador [] y testear Shapiro
    columnSeries = shapiro_cols[column]
    stat, p = shapiro(columnSeries)
   # interpretar
    alpha = 0.05
    if p > alpha:
        msg = 'Gaussiana (no se puede rechazar H0)'
        # crear df con variables sin distribución normal
        df_normal[column]=columnSeries
    else:
        msg = 'No Gaussiana (rechazar H0)'
        # crear df con variables sin distribución normal
        df_nonormal[column]=columnSeries       
    
    result_mat = {'Variable':[column],'n':[len(columnSeries)], 'Estadístico': [stat], 'Valor de p':[p], 'Interpretación':[msg]}
    result = pd.DataFrame(result_mat)
    dfs.append(result)
    full = pd.concat(dfs)

full

In [ ]:
#df.iloc[:, 167:267]

# Tiempo desde ACV a epilepsia

In [ ]:
df['tiempo_ACV_epi']=df['epilepsia_fecha']-df['ACV_fecha']
df['tiempo_ACV_epi'].describe()

# Categoricas

## Agrupacion de variables categoricas 

### Creacion columna Afección cortical

In [ ]:
df['Cortical o subcortical?'].value_counts()

In [ ]:
df['Afección cortical']=df['Cortical o subcortical?'].map({
                                 'Subcortical':'No',
                                 'Cortico-subcortical':'Si',
                                 'Cortical':'Si',
})
df['Afección cortical'].value_counts()

In [ ]:
df.columns.to_list()

## Mapeo de todos los NR como un NO

In [ ]:
df.columns.to_list()

In [ ]:
df[['ESUS_x','ESUS_y']].isna().sum()

In [ ]:
categorical = ['epilepsia_postACV', 
               'genero_x', 'preferencia_manual', #'APP',
                'ant_fliar_epilepsia',
       'HTA', 'DMI', 'DMII', 'DLP', 'arritmia', 'arritmia_tipo', 'CI',
       'ICC', 'ACVi', 'ACVh', 'AIT', 'ant_personal_epilepsia', 'DCL',
       'demencia', 'hipotiroidismo', 'EPOC', 'asma', 'inmunologicas',
       'CA', 'CA_tipo', 'psiquiatricos', 'psiquiatricos_tipo',
       'trastorno_aprendizaje', 'trastorno_conducta', 'SAHOS',
       #otras categoricas
              #otras categoricas
               'r_TPA',
    'sala_inicio_x', 'ESUS_x', 
               'TOAST_x',
 'TOAST_otras',
 'r_TPA','trombectomia',
 'transformacion_hemorragica',
               'antiagregante1',
 'antiagregante2',
 'ACO1',
 'crisis_tempranas',
 'crisis_temprana_DAE1',
 'crisis_temprana_DAE2',
 'craniectomía',
 'craneoplastia',
    'reinternaciones',
 'reinternacion1',
 'reinternacion2',
               'tipo_crisis',
 'frecuencia_crisis',
 'DAE_sin_epilepsia_postACV',
 'DAE1_postACV',
    #imagenes
    'Lado',
 'Cortical o subcortical?',
               'Afección cortical',
 'Infarto único o múltiple?',
 #'Tipo de infarto múltiple',
 'Transformación hemorrágica 2',]


In [ ]:
df[categorical] = df[categorical].replace({'NR':'No'})

# Tabla 1

In [ ]:
#df.iloc[:, 137:267].columns

In [ ]:
#df.drop(['NIHSSi_y', 'mRSi_y', 'NIHSSe_y', 'mRSe_y', 'diasUTI'], axis=1)

In [ ]:
from scipy.stats import shapiro

def shapiro_test (df):
    #Columnas a testear
    shapiro_cols=df.select_dtypes(exclude=['datetime','object'])

    # Crear dataframes vacios
    dfs = []
    df_nonormal=pd.DataFrame()
    df_normal = pd.DataFrame()

    for column in shapiro_cols:
       # seleccionar los contenidos de las columnas con el operador [] y testear Shapiro
        nans=shapiro_cols[column].isnull().sum()
        columnSeries = shapiro_cols[column].dropna(axis=0)
        stat, p = shapiro(columnSeries)
       # interpretar
        alpha = 0.05
        if p > alpha:
            msg = 'Gaussiana (no se puede rechazar H0)'
            # crear df con variables sin distribución normal
            df_normal[column]=columnSeries
        else:
            msg = 'No Gaussiana (rechazar H0)'
            # crear df con variables sin distribución normal
            df_nonormal[column]=columnSeries       

        result_mat = {'Variable':[column],'n':[len(columnSeries)], 'nans':[nans], 
                      'Estadístico': [stat], 'Valor de p':[p], 'Interpretación':[msg]}
        result = pd.DataFrame(result_mat)
        dfs.append(result)
        full = pd.concat(dfs)

    return full

In [ ]:
pd.options.display.max_rows = 300

from tableone import TableOne
#https://github.com/tompollard/tableone/blob/master/tableone.ipynb

# columns to summarize
data = df[[#categoricas
    'edad_y',
       'genero_x', 
    #'ant_fliar_epilepsia',
      'psiquiatricos',# 'psiquiatricos_tipo',
       'TOAST_x',
 'TOAST_otras',
 'r_TPA','trombectomia',
 'crisis_tempranas',
    'tipo_crisis',
 'frecuencia_crisis',
 #'DAE1_postACV',
    'tiempo_seguimiento', 
    'internacion_dias', 
    'NIHSSi_x',
       'NIHSSe_x', 'atorvastatina_evento_dosis', 'rosuvastatina_evento_dosis',
      'Lado',
 'Cortical o subcortical?',
    'Afección cortical',
 'Infarto único o múltiple?',
 'Transformación hemorrágica 2',
    'Clasificación transformación hemorrágica',  'Fazekas','epilepsia_postACV'
]]

# alternative labels
labels={'edad_y':'Edad',
       'genero_x':'Género', 
    'ant_fliar_epilepsia':'Ant. familiar de epilepsia',
      'psiquiatricos':'Ant. psiquiátricos', 'psiquiatricos_tipo':'Tipo de ant. psiquátrico',
               'TOAST_x':'TOAST',
 'TOAST_otras':'TOAST otras',
 'crisis_tempranas':'Crisis tempranas',
    'tipo_crisis':'Tipo de crisis',
 'frecuencia_crisis':"Frecuencia de crisis",
 'DAE1_postACV':'DAE',
    'internacion_dias':'Internación (días)', 
    'NIHSSi_x':'NIHSS al ingreso',
       'NIHSSe_x':'NIHSS al egreso', 'atorvastatina_evento_dosis':'Dosis de atorvastatina (evento índice)', 
        'rosuvastatina_evento_dosis':'Dosis de rosuvastatina (evento índice)', 
      'Lado':'Hemisferio afectado',
 'Cortical o subcortical?':'Topografía',
 'Transformación hemorrágica 2':'Transformación hemorrágica',
        }

#Rename columns
data=data.rename(columns=labels)

# columns to summarize
columns = list(data.columns.values) 

# columns containing categorical variables
categorical = list(data.select_dtypes(include=['object']).columns.values)
  
# non-normal variables
nonnormal = list(shapiro_test(data).loc[shapiro_test(data)['Valor de p']<0.05]['Variable'].values)

# limit the binary variable to a single row
#limit = {}

# set the order of the categorical variables
#order = {}



# set decimal places for age to 0
decimals = {#'JFK_A':0,'
                }

# optionally, a categorical variable for stratification
#groupby = ['epilepsia_postACV']

# create tableone with the input arguments
tabla_totales = TableOne(data, columns=columns, categorical=categorical, 
                   nonnormal=nonnormal, rename=labels, label_suffix=True, 
                   decimals=decimals, #limit=limit, order=order
                        )

tabla_totales

In [ ]:
# optionally, a categorical variable for stratification
groupby = ['epilepsia_postACV']

#Rename colum
# create grouped_table with p values
grouped_table = TableOne(data, columns, categorical, groupby, nonnormal, pval = True, smd=True,
                  htest_name=True)

grouped_table.to_excel('tabla2.xlsx')
#files.download('tabla2.xlsx')
grouped_table

In [ ]:
g = sns.catplot("frecuencia_crisis", col="tipo_crisis", hue='epilepsia_postACV',col_wrap=2, data=df,
                 kind="count", height=4,
                aspect=1)

In [ ]:
pd.options.display.max_rows = 300

from tableone import TableOne
#https://github.com/tompollard/tableone/blob/master/tableone.ipynb

data=df

# columns to summarize
columns = [#categoricas
    'epilepsia_postACV', #'edad_x',
       'genero_x', 'preferencia_manual', #'APP',
    'ant_fliar_epilepsia',
       'HTA', 'DMI', 'DMII', 'DLP', 'arritmia', 'arritmia_tipo', 'CI',
       'ICC', 'ACVi', 'ACVh', 'AIT', 'ant_personal_epilepsia', 'DCL',
       'demencia', 'hipotiroidismo', 'EPOC', 'asma', 'inmunologicas',
       'CA', 'CA_tipo', 'psiquiatricos', 'psiquiatricos_tipo',
       'trastorno_aprendizaje', 'trastorno_conducta', 'SAHOS',
    #otras categoricas
       'TOAST_x',
 'TOAST_otras',
 'r_TPA','trombectomia',
 'transformacion_hemorragica',
 'crisis_tempranas',
 'crisis_temprana_DAE1',
 'crisis_temprana_DAE2',
    'tipo_crisis',
 'frecuencia_crisis',
 'DAE_sin_epilepsia_postACV',
 'DAE1_postACV',
 'craniectomía',
 'craneoplastia',
    'reinternaciones',
 'reinternacion1',
 'reinternacion2',
    #imagenes
    'Lado',
 'Cortical o subcortical?',
    'Afección cortical',
 'Infarto único o múltiple?',
 'Tipo de infarto múltiple',
 'Transformación hemorrágica 2',
    'Clasificación transformación hemorrágica',
       'ACVi localización >> Frontal >> Derecho',
       'ACVi localización >> Frontal >> Izquierdo',
       'ACVi localización >> Frontal >> Bilateral',
       'ACVi localización >> Temporal >> Derecho',
       'ACVi localización >> Temporal >> Izquierdo',
       'ACVi localización >> Parietal >> Derecho',
       'ACVi localización >> Parietal >> Izquierdo',
       'ACVi localización >> Parietal >> Bilateral',
       'ACVi localización >> Occipital >> Derecho',
       'ACVi localización >> Occipital >> Izquierdo',
       'ACVi localización >> Occipital >> Bilateral',
       'ACVi localización >> Capsula interna >> Derecho',
       'ACVi localización >> Capsula interna >> Izquierdo',
       'ACVi localización >> Cerebelo >> Derecho',
       'ACVi localización >> Cerebelo >> Izquierdo',
       'ACVi localización >> Cerebelo >> Bilateral',
       'ACVi localización >> Ganglios de la base >> Derecho',
       'ACVi localización >> Ganglios de la base >> Izquierdo',
       'ACVi localización >> Ganglios de la base >> Bilateral',
       'ACVi localización >> Mesencefalo >> Derecho',
       'ACVi localización >> Mesencefalo >> Izquierdo',
       'ACVi localización >> Protuberancia >> Derecho',
       'ACVi localización >> Protuberancia >> Izquierdo',
       'ACVi localización >> Bulbo >> Derecho',
       'ACVi localización >> Bulbo >> Izquierdo',
       'ACVi localización >> Limítrofe >> Derecho',
       'ACVi localización >> Limítrofe >> Izquierdo',
       'ACVi localización >> Limítrofe >> Bilateral',
       'ACVi localización >> Talamo >> Derecho',
       'ACVi localización >> Talamo >> Izquierdo',
       'ACVi localización >> Insula >> Derecho',
       'ACVi localización >> Insula >> Izquierdo', 'Hipocampo', 
    #numericas
    'tiempo_seguimiento', 'pack/year', 
       'atorvastatina_previa_dosis', 'rosuvastatina_previa_dosis',
       'sinvastatina_previa_dosis', 'internacion_dias', 'guardia_dias',
       'UCV_dias', 'telemetria_dias', 'sala_dias', 'NIHSSi_x', 'mRSi_x',
       'NIHSSe_x', 'atorvastatina_evento_dosis', 'rosuvastatina_evento_dosis',
       'Fazekas', 'edad_y', 
       'diasTotal_x', 'diasUCV_x', 'diasPiso_x',
]

# columns containing categorical variables
categorical = ['epilepsia_postACV', 
       'genero_x', 'preferencia_manual', #'APP',
    'ant_fliar_epilepsia',
       'HTA', 'DMI', 'DMII', 'DLP', 'arritmia', 'arritmia_tipo', 'CI',
       'ICC', 'ACVi', 'ACVh', 'AIT', 'ant_personal_epilepsia', 'DCL',
       'demencia', 'hipotiroidismo', 'EPOC', 'asma', 'inmunologicas',
       'CA', 'CA_tipo', 'psiquiatricos', 'psiquiatricos_tipo',
       'trastorno_aprendizaje', 'trastorno_conducta', 'SAHOS',
       #otras categoricas
              #otras categoricas
    'TOAST_x',
 'TOAST_otras',
 'r_TPA','trombectomia',
 'transformacion_hemorragica',
 'crisis_tempranas',
 'crisis_temprana_DAE1',
 'crisis_temprana_DAE2',
 'craniectomía',
 'craneoplastia',
    'reinternaciones',
 'reinternacion1',
 'reinternacion2',
    #imagenes
    'Lado',
 'Cortical o subcortical?',
               'Afección cortical',
 'Infarto único o múltiple?',
 'Tipo de infarto múltiple',
 'Transformación hemorrágica 2',
              'Clasificación transformación hemorrágica',
       'ACVi localización >> Frontal >> Derecho',
       'ACVi localización >> Frontal >> Izquierdo',
       'ACVi localización >> Frontal >> Bilateral',
       'ACVi localización >> Temporal >> Derecho',
       'ACVi localización >> Temporal >> Izquierdo',
       'ACVi localización >> Parietal >> Derecho',
       'ACVi localización >> Parietal >> Izquierdo',
       'ACVi localización >> Parietal >> Bilateral',
       'ACVi localización >> Occipital >> Derecho',
       'ACVi localización >> Occipital >> Izquierdo',
       'ACVi localización >> Occipital >> Bilateral',
       'ACVi localización >> Capsula interna >> Derecho',
       'ACVi localización >> Capsula interna >> Izquierdo',
       'ACVi localización >> Cerebelo >> Derecho',
       'ACVi localización >> Cerebelo >> Izquierdo',
       'ACVi localización >> Cerebelo >> Bilateral',
       'ACVi localización >> Ganglios de la base >> Derecho',
       'ACVi localización >> Ganglios de la base >> Izquierdo',
       'ACVi localización >> Ganglios de la base >> Bilateral',
       'ACVi localización >> Mesencefalo >> Derecho',
       'ACVi localización >> Mesencefalo >> Izquierdo',
       'ACVi localización >> Protuberancia >> Derecho',
       'ACVi localización >> Protuberancia >> Izquierdo',
       'ACVi localización >> Bulbo >> Derecho',
       'ACVi localización >> Bulbo >> Izquierdo',
       'ACVi localización >> Limítrofe >> Derecho',
       'ACVi localización >> Limítrofe >> Izquierdo',
       'ACVi localización >> Limítrofe >> Bilateral',
       'ACVi localización >> Talamo >> Derecho',
       'ACVi localización >> Talamo >> Izquierdo',
       'ACVi localización >> Insula >> Derecho',
       'ACVi localización >> Insula >> Izquierdo', 'Hipocampo','tipo_crisis','frecuencia_crisis',
               'DAE_sin_epilepsia_postACV', 'DAE1_postACV']
  
# non-normal variables
nonnormal = [#'edad_x',
             'tiempo_seguimiento', 'pack/year', 'atorvastatina_previa_dosis',
       'rosuvastatina_previa_dosis', 'sinvastatina_previa_dosis',
       'internacion_dias', 'guardia_dias', 'UCV_dias', 'telemetria_dias',
       'sala_dias', 'NIHSSi_x', 'mRSi_x', 'NIHSSe_x',
       'atorvastatina_evento_dosis', 'rosuvastatina_evento_dosis', 'Fazekas',
       'edad_y', 'NIHSSi_y', 'mRSi_y', 'NIHSSe_y', 'mRSe_y', 'diasTotal',
       'diasUCV', 'diasUTI', 'diasPiso']

# limit the binary variable to a single row
#limit = {}

# set the order of the categorical variables
#order = {}

# alternative labels
labels={#'genero':'Género', 
        }

# set decimal places for age to 0
decimals = {#'JFK_A':0,'
                }

# optionally, a categorical variable for stratification
groupby = ['epilepsia_postACV']

# create tableone with the input arguments
tabla_totales = TableOne(data, columns=columns, categorical=categorical, 
                   nonnormal=nonnormal, rename=labels, label_suffix=True, 
                   decimals=decimals, #limit=limit, order=order
                        )

tabla_totales

In [ ]:
# create grouped_table with p values
grouped_table = TableOne(df, columns, categorical, groupby, 
                         nonnormal, labels=labels, decimals=decimals, label_suffix=True, 
                         pval = True, smd=True, htest_name=True)
grouped_table

# Correlaciones

In [ ]:
df['psiquiatricos_cat']=df['psiquiatricos'].map({'No':0, 'Si':1, 'NR':0})
df['trombectomia_cat']=df['trombectomia'].map({'No':0, 'Si':1,'NR':0})
df['transformacion_hemorragica_cat']=df['Transformación hemorrágica 2'].map({'No':0, 'Si':1})
df['epilepsia_postACV_cat']=df['epilepsia_postACV'].map({'No':0, 'Si':1})
df['crisis_tempranas_cat']=df['crisis_tempranas'].map({'No':0, 'Si':1, 'NR':0})




df_corr=df[['psiquiatricos_cat','trombectomia_cat', 'transformacion_hemorragica_cat','crisis_tempranas_cat',
            'NIHSSi_x','internacion_dias',
         'epilepsia_postACV_cat']]

In [ ]:
plt.figure(figsize=(15,7))
plt.title('Correlaciónes de Spearman', y=1.05, size=15)
sns.heatmap(df_corr.corr(method='spearman'), annot=True, square=True, #cmap=plt.cm.Reds
        )
plt.show()

In [ ]:
from scipy.stats import kendalltau, pearsonr, spearmanr
def kendall_pval(x,y):
    return kendalltau(x,y)[1]
def pearsonr_pval(x,y):
    return pearsonr(x,y)[1]
def spearmanr_pval(x,y):
    return spearmanr(x,y)[1]

spearman_p=df_corr.corr(method=spearmanr_pval)
spearman_p=spearman_p.loc[spearman_p['epilepsia_postACV_cat'] <= 0.05]
print(spearman_p.index.values.tolist())

In [ ]:
spearman_p

In [ ]:
# Funcion para calcular la correlacion
# Nos permite usar pearson o spearman, y devuelve p value
from scipy.stats import pearsonr, spearmanr

def correlate(df, corr=pearsonr):
    matrix = df.values.T
    rows, cols = matrix.shape[0], matrix.shape[1]
    r = np.ones(shape=(rows, rows))
    p = np.ones(shape=(rows, rows))
    for i in range(rows):
        for j in range(i+1, rows):
            r_, p_ = corr(matrix[i], matrix[j])
            r[i, j] = r[j, i] = r_
            p[i, j] = p[j, i] = p_
    return r, p

In [ ]:
cols=df[['psiquiatricos_cat','trombectomia_cat', 'transformacion_hemorragica_cat','crisis_tempranas_cat',
            'NIHSSi_x','internacion_dias',
         'epilepsia_postACV_cat']].columns
#cols = df.select_dtypes(include=[np.number]).columns

corrs, ps = correlate(df[cols], spearmanr)

# columna a correlacionar con las demás
col_corr = 'epilepsia_postACV_cat'  
col_corr_idx = cols.get_loc(col_corr)

# guardamos las correlaciones en un dataframe
corr_df = pd.DataFrame(corrs[col_corr_idx], cols, columns=['coef'])
corr_df['p'] = ps[col_corr_idx]

corr_df = corr_df.iloc[corr_df['coef'].abs().argsort().values[::-1], :]
corr_df[(np.abs(corr_df['coef']) < 0) & (corr_df['p'] < 1e-5)]

# Análisis multivariable

In [ ]:
import statsmodels.api as sm
import scipy.stats as st

logit_X=df[['psiquiatricos_cat','trombectomia_cat', 'transformacion_hemorragica_cat','crisis_tempranas_cat',
            'internacion_dias','NIHSSi_x','NIHSSe_x','Fazekas','rosuvastatina_evento_dosis'
           ]]
logit_y=df[['epilepsia_postACV_cat']]


#cols=logit_df_constant.columns[:-1]
model=sm.Logit(logit_y,logit_X)
result=model.fit(method='cg', maxiters=1000) #methosd: 'bfgs', 'nm', 'newton' fit_regularized(method='l1')
result.summary()

In [ ]:
def back_feature_elem (data_frame,dep_var,col_list, reg=False, method=None):
    """ Takes in the dataframe, the dependent variable and a list of column names, runs the regression repeatedly eleminating feature with the highest
    P-value above alpha one at a time and returns the regression summary with all p-values below alpha"""

    while len(col_list)>0 :
        model=sm.Logit(dep_var,data_frame[col_list], method=method)
        if reg:
          result=model.fit_regularized(disp=False)
        else:
          result=model.fit()
        largest_pvalue=round(result.pvalues,6).nlargest(1)
        if largest_pvalue[0]<(0.05):
            return result
            break
        else:
            col_list=col_list.drop(largest_pvalue.index)

result=back_feature_elem(logit_X, logit_y, logit_X.columns, reg=True)
result.summary()

In [ ]:
params = np.exp(result.params)
conf = np.exp(result.conf_int())
conf['OR'] = params
pvalue=round(result.pvalues,6)
conf['pvalue']=pvalue
conf.columns = ['CI 95%(2.5%)', 'CI 95%(97.5%)', 'Odds Ratio','pvalue']
print ((conf))